# ТМО РК №2 
### Шиленок Даниил Андреевич ИУ5-65Б
##### Вариант 20

|Группа|Метод №1|Метод №2|
|-|-|-|
|ИУ5-65Б|Метод опорных векторов|Градиентный бустинг|

Задание. Для заданного набора данных (по Вашему варианту) постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы 1 и 2 (по варианту для Вашей группы). Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

Для данного датасета построим модель для решения задачи классификации.

В качестве целевого признака выберем koi_disposition (CANDIDATE, FALSE POSITIVE, CONFIRMED)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

data = pd.read_csv('cumulative.csv')

In [88]:
print(data.columns)

Index(['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol',
       'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_tce_delivname', 'koi_steff', 'koi_steff_err1', 'koi_steff_err2',
       'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad',
       'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')


In [89]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rowid              9564 non-null   int64  
 1   kepid              9564 non-null   int64  
 2   kepoi_name         9564 non-null   object 
 3   kepler_name        2294 non-null   object 
 4   koi_disposition    9564 non-null   object 
 5   koi_pdisposition   9564 non-null   object 
 6   koi_score          8054 non-null   float64
 7   koi_fpflag_nt      9564 non-null   int64  
 8   koi_fpflag_ss      9564 non-null   int64  
 9   koi_fpflag_co      9564 non-null   int64  
 10  koi_fpflag_ec      9564 non-null   int64  
 11  koi_period         9564 non-null   float64
 12  koi_period_err1    9110 non-null   float64
 13  koi_period_err2    9110 non-null   float64
 14  koi_time0bk        9564 non-null   float64
 15  koi_time0bk_err1   9110 non-null   float64
 16  koi_time0bk_err2   9110 

In [90]:
data.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [91]:
# Удаление ненужных столбцов
columns_to_drop = [
    'rowid', 'kepid', 'kepoi_name', 'kepler_name',
    'koi_tce_delivname', 'koi_tce_plnt_num',
    'koi_score', 'koi_pdisposition', 'koi_fpflag_nt', 'koi_fpflag_ss',
    'koi_fpflag_co', 'koi_fpflag_ec'
]
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])

In [92]:
print(data.isnull().sum())

koi_disposition         0
koi_period              0
koi_period_err1       454
koi_period_err2       454
koi_time0bk             0
koi_time0bk_err1      454
koi_time0bk_err2      454
koi_impact            363
koi_impact_err1       454
koi_impact_err2       454
koi_duration            0
koi_duration_err1     454
koi_duration_err2     454
koi_depth             363
koi_depth_err1        454
koi_depth_err2        454
koi_prad              363
koi_prad_err1         363
koi_prad_err2         363
koi_teq               363
koi_teq_err1         9564
koi_teq_err2         9564
koi_insol             321
koi_insol_err1        321
koi_insol_err2        321
koi_model_snr         363
koi_steff             363
koi_steff_err1        468
koi_steff_err2        483
koi_slogg             363
koi_slogg_err1        468
koi_slogg_err2        468
koi_srad              363
koi_srad_err1         468
koi_srad_err2         468
ra                      0
dec                     0
koi_kepmag              1
dtype: int64

In [93]:
# Обработка пропусков
# Сначала проверим, какие столбцы полностью пустые
empty_cols = data.columns[data.isna().all()].tolist()
if empty_cols:
    print(f"Удаляем полностью пустые столбцы: {empty_cols}")
    data = data.drop(columns=empty_cols)

Удаляем полностью пустые столбцы: ['koi_teq_err1', 'koi_teq_err2']


In [94]:
# Числовые признаки
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'koi_disposition']
# Категориальные признаки
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()

# Заполнение пропусков только в тех столбцах, где есть хотя бы одно значение
numeric_cols_to_impute = [col for col in numeric_cols if data[col].notna().any()]
if numeric_cols_to_impute:
    imputer_num = SimpleImputer(strategy='median')
    data[numeric_cols_to_impute] = imputer_num.fit_transform(data[numeric_cols_to_impute])

categorical_cols_to_impute = [col for col in categorical_cols if data[col].notna().any()]
if categorical_cols_to_impute:
    imputer_cat = SimpleImputer(strategy='most_frequent')
    data[categorical_cols_to_impute] = imputer_cat.fit_transform(data[categorical_cols_to_impute])

# Кодирование целевой переменной
label_encoder = LabelEncoder()
data['koi_disposition'] = label_encoder.fit_transform(data['koi_disposition'])

# Удаление оставшихся строковых признаков перед обучением
data = data.select_dtypes(exclude=['object'])

# Проверка на наличие NaN после всех преобразований
if data.isna().any().any():
    print("В данных остались пропущенные значения")
    data = data.dropna()

In [95]:
# Разделение на признаки и целевую переменную
X = data.drop(columns=['koi_disposition'])
y = data['koi_disposition']
# Разделение на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

# Масштабирование числовых признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [96]:
# Метод опорных векторов (SVM)
svm = SVC(probability=True)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

# Градиентный бустинг
gb = GradientBoostingClassifier(random_state=21)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

# Оценка качества
print("Метод опорных векторов:")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("F1-score:", f1_score(y_test, y_pred_svm, average='weighted'))

print("\nГрадиентный бустинг:")
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("F1-score:", f1_score(y_test, y_pred_gb, average='weighted'))


Метод опорных векторов:
Accuracy: 0.7349712493465761
F1-score: 0.7014390173044057

Градиентный бустинг:
Accuracy: 0.7877679038159958
F1-score: 0.7790239010057025
